In [1]:
import torch

import data
import utils
import info_recorder as ir
import data_loader as dl
import initializer as init
import trainer as tn
import tester as ts

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1070


In [3]:
df, size_info, size_dict = data.get_df()

In [4]:
sourceCode_np = df.sourceCode.values
codeClass_np = df.classLabel.values

In [5]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_sourceCodes, ch2idx, max_len = utils.tokenize(sourceCode_np)
input_ids = utils.encode(tokenized_sourceCodes, ch2idx, max_len)

Tokenizing...



In [6]:
ir.record_ch2idx(ch2idx)

In [7]:
encoded_class2idx, class2idx, num_classes = utils.tokenize_encode_class(codeClass_np)

In [8]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids, encoded_class2idx, test_size = 0.1, random_state = 43
)

In [9]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    train_inputs, train_labels, test_size = 0.1, random_state = 43
)

In [10]:
# Load data to PyTorch DataLoader
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_inputs, val_inputs, test_inputs, train_labels, val_labels, test_labels, batch_size=50)

In [11]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('dataDivided/tests')

In [12]:
# CNN-rand: Word vectors are randomly initialized.
tn.set_seed(42)
cnn_rand, optimizer = init.initilize_model(device=device,
                                           vocab_size=len(ch2idx),
                                           embed_dim=50,
                                           hidden_size=100,
                                           num_classes=len(class2idx),
                                           n_layers=3,
                                           dropout=0.2,
                                           learning_rate=0.25,
                                           optimizerName="Adadelta",
                                           modelType="RNN")
                                        
tn.train(device, cnn_rand, optimizer, train_dataloader, 'DataDiv_test1', writer, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   2.932014   |  2.740202  |   15.52   |  184.31  
   2    |   2.526522   |  2.315258  |   27.05   |  186.85  
   3    |   2.237375   |  2.456015  |   24.04   |  187.25  
   4    |   2.087361   |  1.946706  |   39.23   |  187.34  
   5    |   1.881744   |  1.738022  |   46.45   |  187.39  
   6    |   1.653996   |  1.525561  |   53.64   |  187.39  
   7    |   1.379594   |  1.235324  |   63.63   |  187.46  
   8    |   1.131639   |  0.998926  |   70.87   |  187.46  
   9    |   0.948364   |  0.886966  |   74.04   |  187.46  
  10    |   0.831930   |  0.792443  |   76.89   |  187.41  
  11    |   0.750742   |  0.723527  |   78.85   |  187.40  
  12    |   0.685795   |  0.682930  |   80.24   |  187.41  
  13    |   0.638014   |  0.667732  |   80.62   |  187.46  
  14    |   0.598268   |  0.642102  |   81.34   |  187.43  
  15    |   0.563775

In [13]:
ts.test(device, cnn_rand, test_dataloader)

test loss:  0.5851330412463063
test acc:  83.62857142857143
